In [3]:
from SpiderV2 import Spider
from IndexV2 import Database, Index, RawInfoIndex
from Tokenize import Tokenize
from Searcher import Search
import time

In [4]:
db = Database(SearchEngine="ForSpiderTest")
index = Index()
raw_storage = RawInfoIndex()
tokenizer = Tokenize()
raw_storage.read_from_database(db)
index.read_fw_index_from_database(db)
index.read_ivi_index_from_database(db)

In [15]:

url = "https://www.thairath.co.th/news/local/bangkok/2635906"
#"https://en.wikipedia.org/wiki/Climate_change", "https://riskofrain.fandom.com/wiki/Main_Page", "https://www.thairath.co.th/news/local/bangkok/2635906" 
Spider.set_base_domains(["https://www.thairath.co.th/", "https://www.thairath.co.th/", "https://riskofrain.fandom.com/wiki/Main_Page", "https://en.wikipedia.org/", "https://iot-kmutnb.github.io/", "https://riskofrain2.fandom.com/"])
print(Spider.all_robots)
print(Spider.queue)
Spider.queue_front = 0
Spider.queue_back = 1
Spider.depth = 0

spider = Spider(url)
success = spider.generate_next_soup()

raw_storage.read_from_database(db)

if success:
    info = spider.crawl({})
    raw_storage.modify_index(Spider.queue[Spider.queue_front-1], info[0], info[1], info[2])
    raw_storage.save_to_database(db)
else:
    if Spider.get_base_domain(url) in Spider.all_robots:
        Spider.unaccessible_urls.append(url)
    print("Cannot crawl to the website.")

{'www.thairath.co.th': <urllib.robotparser.RobotFileParser object at 0x000001559C46EF20>, 'riskofrain.fandom.com': <urllib.robotparser.RobotFileParser object at 0x00000155C49C8430>, 'en.wikipedia.org': <urllib.robotparser.RobotFileParser object at 0x000001559C46ED70>, 'iot-kmutnb.github.io': None, 'riskofrain2.fandom.com': <urllib.robotparser.RobotFileParser object at 0x000001559C46DA20>}
['https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki', 'https://www.fandom.com/what-is-fandom', 'https://riskofrain2.fandom.com/wiki/Commencement', 'https://bit.ly/TikTokFandom', 'https://twitter.com/RiskofRain', 'https://store.steampowered.com/app/632360/Risk_of_Rain_2/', 'https://riskofrain2.fandom.com/wiki/Artificer', 'https://riskofrain2.fandom.com/wiki/Items', 'https://www.fandom.com/about', 'https://www.fandom.com/topics/tv', 'https://riskofrain2.fandom.com/wiki/Abandoned_Aqueduct', 'http://www.gearboxpublishing.com/', 'https://riskofrain2.fandom.com/wiki/Abyssal_Depths', 'https://riskofrai

In [8]:
for url in raw_storage.index:
    tokens = tokenizer.tokenize(raw_storage.index[url]["text"])
    tokens = tokenizer.filter(tokens)
    tokens = tokenizer.make_counter(tokens)
    index.modify_index(url, tokens)
raw_storage.save_to_database(db)
index.save_fw_index_to_database(db)
index.save_ivi_index_to_database(db)

In [5]:
print(raw_storage.get_ref_count("https://riskofrain.fandom.com/wiki/Main_Page"))

1


In [7]:
raw_storage.remove_urls(["https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki"])
index.remove_urls(raw_storage.url_to_be_deleted)

In [9]:
print(index.ivi_index["risk"])

{'https://en.wikipedia.org/wiki/Climate_change': 39, 'https://riskofrain.fandom.com/wiki/Main_Page': 26, 'https://riskofrain2.fandom.com/wiki/Risk_of_Rain_2_Wiki': 18}


In [10]:
print(Search().search(["arduino", "and", "python"], raw_storage.index, index.fw_index, index.ivi_index))

[('https://iot-kmutnb.github.io/blogs', 0.08881312865035303, 0, {})]
